In [61]:
from transform import pick_date, filter_date, get_age, substract_months
import pandas as pd
import numpy as np

In [62]:
df_applicant_education = pd.read_csv('data/df_applicant_education.csv', index_col=['ApplicantEducationID'])
df_applicant_education.drop(columns=['Unnamed: 0'], inplace=True)
df_applicant_education.drop_duplicates(['ApplicantID', 'DateStart', 'DateEnd'], inplace=True)

In [63]:
df_applicant_education.head(3)

,ApplicantID,DateStart,DateEnd,EducationInstituteName,Score,EducationLevelID,MajorID,Degree
ApplicantEducationID,,,,,,,,
1,3,2011-07-01 00:00:00.0000000,2016-01-01 00:00:00.0000000,ATMAJAYA YOGYAKARTA UNIVERSITY,3.01,2,7.0,True
2,4,2013-07-01 00:00:00.0000000,2017-10-01 00:00:00.0000000,UNIVERSITAS MUHAMMADIYAH SIDOARJO,3.37,2,4.0,True
3,8,2012-10-01 00:00:00.0000000,2016-02-01 00:00:00.0000000,STIE PERBANAS MALANG,328.00,2,7.0,True


In [64]:
df_applicant_education.DateStart = df_applicant_education.DateStart.map(pick_date).apply(lambda x: filter_date(x, 1980, 2023))
df_applicant_education.DateEnd = df_applicant_education.DateEnd.map(pick_date).apply(lambda x: filter_date(x, 1980, 2023))

In [65]:
df_applicant_education.head(3)

,ApplicantID,DateStart,DateEnd,EducationInstituteName,Score,EducationLevelID,MajorID,Degree
ApplicantEducationID,,,,,,,,
1,3,2011-07-01,2016-01-01,ATMAJAYA YOGYAKARTA UNIVERSITY,3.01,2,7.0,True
2,4,2013-07-01,2017-10-01,UNIVERSITAS MUHAMMADIYAH SIDOARJO,3.37,2,4.0,True
3,8,2012-10-01,2016-02-01,STIE PERBANAS MALANG,328.00,2,7.0,True


In [66]:
df_applicant_education.DateStart = pd.to_datetime(df_applicant_education.DateStart)
df_applicant_education.DateEnd = pd.to_datetime(df_applicant_education.DateEnd)

In [67]:
df_applicant_education.DateStart[
    (df_applicant_education.DateStart.isnull()) | (df_applicant_education.DateEnd.isnull())
] = pd.NaT
df_applicant_education.DateEnd[
    (df_applicant_education.DateStart.isnull()) | (df_applicant_education.DateEnd.isnull())
] = pd.NaT

C:\Users\febri\AppData\Local\Temp\ipykernel_13664\654398753.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_applicant_education.DateStart[
C:\Users\febri\AppData\Local\Temp\ipykernel_13664\654398753.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_applicant_education.DateEnd[


In [68]:
df_applicant_education['YearsOfEducation'] = substract_months(df_applicant_education.DateStart, df_applicant_education.DateEnd).apply(lambda x: int(x/12) if (x > 1) else 0)

In [69]:
df_applicant_education.tail(3)

,ApplicantID,DateStart,DateEnd,EducationInstituteName,Score,EducationLevelID,MajorID,Degree,YearsOfEducation
ApplicantEducationID,,,,,,,,,
15499,44928,2010-05-31,2013-05-23,SMA Kristen YPKPM Ambon,6.90,5,13.0,True,3
15500,44929,2010-09-12,2014-10-29,Universitas Gunadarma,3.16,2,4.0,True,4
15501,44930,2012-09-14,2008-08-17,Universitas Islam Jakarta,3.36,2,7.0,True,0


In [70]:
df_applicant_education.YearsOfEducation = df_applicant_education.YearsOfEducation.apply(lambda x: x if x <= 7 else 0)

In [87]:
df_applicant_education.head(2)

,ApplicantID,DateStart,DateEnd,EducationInstituteName,Score,EducationLevelID,MajorID,Degree,YearsOfEducation
ApplicantEducationID,,,,,,,,,
1,3,2011-07-01,2016-01-01,ATMAJAYA YOGYAKARTA UNIVERSITY,3.01,2,7.0,True,4
2,4,2013-07-01,2017-10-01,UNIVERSITAS MUHAMMADIYAH SIDOARJO,3.37,2,4.0,True,4


In [88]:
df_applicant_education = df_applicant_education.sort_values('DateStart').groupby(['ApplicantID']).agg({
    'DateStart': 'first',
    'DateEnd': 'last',
    'EducationInstituteName': 'last',
    'Score': 'last',
    'EducationLevelID': 'last',
    'MajorID': 'last',
    'Degree': 'last',
    'YearsOfEducation': 'sum',
})

In [96]:
432/10*4

172.8

In [98]:
95/100*4

3.8

In [110]:
def convert_gpa(val: float):
    if 0 < val <= 1000:
        if val <= 4:
            return val
        elif 4 < val <= 10:
            return val/10*4
        elif 10 < val <= 40:
            return val/10
        elif 40 < val <= 100:
            return val/100*4
        elif 100 < val <= 400:
            return val/400
        else:
            return val/1000*4
    else:
        return 0

In [114]:
df_applicant_education.Score = df_applicant_education.Score.map(convert_gpa)

In [116]:
df_applicant_education.head(2)

,DateStart,DateEnd,EducationInstituteName,Score,EducationLevelID,MajorID,Degree,YearsOfEducation
ApplicantID,,,,,,,,
1,2011-08-01,2014-08-01,SMAN 16 SURABAYA,2.00,5,13.0,False,3
3,2011-07-01,2016-01-01,ATMAJAYA YOGYAKARTA UNIVERSITY,3.01,2,7.0,True,4


In [123]:
df_applicant_education.MajorID = df_applicant_education.MajorID.fillna(0).astype(int)

In [125]:
df_applicant_education.head(2)

,DateStart,DateEnd,EducationInstituteName,Score,EducationLevelID,MajorID,Degree,YearsOfEducation
ApplicantID,,,,,,,,
1,2011-08-01,2014-08-01,SMAN 16 SURABAYA,2.00,5,13,False,3
3,2011-07-01,2016-01-01,ATMAJAYA YOGYAKARTA UNIVERSITY,3.01,2,7,True,4


In [127]:
df_applicant_education.drop(columns=['DateStart', 'DateEnd'], inplace=True)

In [128]:
df_applicant_education.EducationInstituteName = df_applicant_education.EducationInstituteName.map(str.lower)

In [129]:
df_applicant_education.head(3)

,EducationInstituteName,Score,EducationLevelID,MajorID,Degree,YearsOfEducation
ApplicantID,,,,,,
1,sman 16 surabaya,2.00,5,13,False,3
3,atmajaya yogyakarta university,3.01,2,7,True,4
4,universitas muhammadiyah sidoarjo,3.37,2,4,True,4


In [130]:
df_applicant_education.to_csv('data/cdf_applicant_education.csv')